In [1]:
#Import thư viện cần thiết
from pdf2image import convert_from_path
import os
import re
import unicodedata
from collections import defaultdict
import difflib
import pandas as pd
from PIL import Image
import pytesseract
import cv2
import numpy as np
import matplotlib.pyplot as plt
from paddleocr import PaddleOCR
import torch
from sentence_transformers import SentenceTransformer, util
from tqdm import tqdm
from sacrebleu.metrics import BLEU
from bert_score import score
from googletrans import Translator
import time

1. Chuyển file PDF sang các file ảnh

In [4]:
#Xử lí PDF 1 thành hình ảnh dạng png từ trang 203 đến trang 395 (theo id)
pdf_file = "../data/pdf/PDF1.pdf"
output_folder = "../image/PDF1"


images = convert_from_path(
    pdf_file,
    first_page=203,
    last_page=395
)

for page_num, img in enumerate(images, start=203):
    filename = f"PDF1-{page_num}.png"
    save_path = os.path.join(output_folder, filename)
    img.save(save_path, "PNG")


In [5]:
#Xử lí PDF 3 thành hình ảnh dạng png từ trang 12 đến trang 118 (theo id)
pdf_file = "../data/pdf/PDF3.pdf"
output_folder = "../image/PDF3"

os.makedirs(output_folder, exist_ok=True)

start_page = 12
end_page = 118

images = convert_from_path(pdf_file, first_page=start_page, last_page=end_page)

for page_num, img in zip(range(start_page, end_page + 1), images):
    page_str = f"{page_num:03d}"           
    filename = f"PDF3-{page_str}.png"
    save_path = os.path.join(output_folder, filename)

    img.save(save_path, "PNG")

2. Tiền xử lí ảnh trước khi OCR

In [2]:
# =====================================================
INPUT_ROOT = '../image'
INPUT_SUBDIR = 'PDF1'
INPUT_DIR = os.path.join(INPUT_ROOT, INPUT_SUBDIR)

OUTPUT_SUBDIR = 'PDF1_processed'
OUTPUT_DIR = os.path.join(INPUT_ROOT, OUTPUT_SUBDIR)

START_PAGE = 250
END_PAGE = 296

if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)
    print(f"✅ Đã tạo thư mục đầu ra: {OUTPUT_DIR}")

# =====================================================
# 2. LỌC FILE THEO SỐ TRANG (GIỮ NGUYÊN)
# =====================================================
target_files = []
pattern = re.compile(r'PDF1-(\d+)\.png$', re.IGNORECASE)

try:
    all_files = os.listdir(INPUT_DIR)
    for file_name in all_files:
        match = pattern.match(file_name)
        if match:
            page_number = int(match.group(1))
            if START_PAGE <= page_number <= END_PAGE:
                target_files.append(file_name)
except FileNotFoundError:
    print(f" Không tìm thấy thư mục đầu vào: {INPUT_DIR}")

target_files.sort(key=lambda f: int(pattern.search(f).group(1)))

# =====================================================
# 3. HÀM XỬ LÝ TỐI ƯU OCR: CHỐNG MẤT NÉT & LÀM ĐẬM CHỮ
# =====================================================
def process_optimized_for_ocr(image_path):
    img = cv2.imread(image_path)
    if img is None:
        return None, None

    # 1. Chuyển sang Grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # 2. Tăng cường độ tương phản cục bộ (CLAHE)
    # Giúp các dòng chữ nhạt màu như "số 641" hiện rõ hơn khỏi nền
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
    enhanced = clahe.apply(gray)

    # 3. Adaptive Thresholding (Phân ngưỡng thích nghi)
    # Thay vì dùng ngưỡng cố định cho cả ảnh, nó tính toán từng vùng nhỏ
    # giúp giữ lại nét chữ mảnh cực tốt.
    binary = cv2.adaptiveThreshold(
        enhanced, 255, 
        cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
        cv2.THRESH_BINARY, 21, 15
    )

    # 4. KHỬ NHIỄU NHẸ (Dùng Median Blur kích thước nhỏ nhất)
    # Để xóa các đốm đen li ti mà không làm hỏng cấu trúc chữ
    denoised = cv2.medianBlur(binary, 3)

    # 5. LÀM ĐẬM NÉT CHỮ (Erosion)
    # Với ảnh nền trắng chữ đen, phép Erode sẽ làm các vùng đen (chữ) dày lên.
    # Điều này cực kỳ quan trọng để Tesseract nhận diện được các chữ bị nhạt/đứt nét.
    kernel = np.ones((2, 2), np.uint8)
    final_img = cv2.erode(denoised, kernel, iterations=1)

    return final_img, img

# =====================================================
# 4. THỰC THI XỬ LÝ ĐỒNG LOẠT
# =====================================================

count = 0
for file_name in target_files:
    full_path = os.path.join(INPUT_DIR, file_name)
    processed_img, _ = process_optimized_for_ocr(full_path)

    if processed_img is not None:
        output_path = os.path.join(OUTPUT_DIR, f"processed_{file_name}")
        cv2.imwrite(output_path, processed_img)
        count += 1
        if count % 10 == 0:
            print(f" Đã xử lý {count}/{len(target_files)} ảnh")

print(f"\n  Đã lưu {count} ảnh vào: {OUTPUT_DIR}")

 Đã xử lý 10/47 ảnh
 Đã xử lý 20/47 ảnh
 Đã xử lý 30/47 ảnh
 Đã xử lý 40/47 ảnh

  Đã lưu 47 ảnh vào: ../image\PDF1_processed


3. Xử lí các ảnh thông qua sử dụng Paddle OCR và Teserract OCR. 

In [8]:
# OCR Tesseract
TESSERACT_LANGS = 'chi_sim+chi_tra+vie'
def tesseract_ocr(img_path: str, lang: str = TESSERACT_LANGS):
    try:
        text_raw = pytesseract.image_to_string(
            Image.open(img_path),
            lang=lang,
            config='--psm 3'
        )
        return [l.strip() for l in text_raw.split('\n') if l.strip()]
    except Exception as e:
        print(f"Lỗi OCR {img_path}: {e}")
        return []
    
# OCR Paddle
ocr_paddle = PaddleOCR(
    use_angle_cls=True,
    lang='ch',
    show_log=False
)

In [9]:
def strip_accents(s: str) -> str:
    return "".join(
        c for c in unicodedata.normalize("NFD", s)
        if unicodedata.category(c) != "Mn"
    )

def is_chinese(text: str) -> bool:
    if not text:
        return False
        
    pattern = (
        r'^['
        # Nhóm Hán tự
        r'\u4E00-\u9FFF'  # CJK Unified Ideographs (Phổ thông)
        r'\u3400-\u4DBF'  # CJK Extension A
        r'\uF900-\uFAFF'  # CJK Compatibility
        
        # Nhóm dấu câu CJK
        r'\u3000-\u303F'  # Dấu câu TQ (。 、 【 】...)
        r'\uFF00-\uFFEF'  # Dấu câu Full-width (！, ？, ０-９...)
        r'\uFE30-\uFE4F'  # CJK Compatibility Forms
        
        # Nhóm ASCII (không có kí tự Latin)
        r'\u0020-\u0040'  # Khoảng trắng, Dấu câu (!..@), SỐ (0-9)
        r'\u005B-\u0060'  # Dấu câu ( [ .. ` )
        r'\u007B-\u007E'  # Dấu câu ( { .. ~ )
        
        # Nhóm Latin-1 Suplement (Giữ lại để bắt dấu » « ©)
        # r'\u00A0-\u00FF'  
        r']+$'
    )
    
    return bool(re.fullmatch(pattern, text))

def is_page_number_line(text: str) -> bool:
    return re.fullmatch(r"^\s*\d{1,4}\s*$", text) is not None

def looks_like_vietnamese(text: str) -> bool:
    norm = strip_accents(text).lower()
    tokens = re.findall(r"[a-z0-9]+", norm)

    def has_similar(target: str, thr: float) -> bool:
        for t in tokens:
            if difflib.SequenceMatcher(None, t, target).ratio() >= thr:
                return True
        return False

    return (
        has_similar("buc", 0.5) and
        has_similar("thu", 0.5) and
        has_similar("viet", 0.5) and
        has_similar("cho", 0.5) and
        has_similar("chinh", 0.5) and
        has_similar("minh", 0.5)
    )

def extract_id(text: str):
    def norm_digits(tok: str):
        tok = tok.strip()
        tok = tok.replace('O', '0').replace('o', '0')
        tok = tok.replace('I', '1').replace('l', '1').replace('|', '1').replace('!', '1')

        digits = "".join(re.findall(r"\d+", tok))  # "50 4" -> "504"
        if not digits:
            return None
        if len(digits) < 3:
            return None
        return digits

    # Chinese: 第xxx 
    m = re.search(r"第\s*([0-9OoIl|!\s]{1,20})", text)
    if m:
        return norm_digits(m.group(1))

    # Vietnamese: so/s0/s6 ...
    norm = strip_accents(text).lower()
    m = re.search(r"(?:\bso\b|\bs0\b|\bs6\b|s[0o6])\s*([0-9OoIl|!\s]{1,20})", norm)
    if m:
        return norm_digits(m.group(1))

    return None

In [10]:
# Parse ảnh -> Data
DATA_MAP_TESSERACT = defaultdict(lambda: {"src": [], "tgt": [], "vi_started": False})
DATA_MAP_PADDLE = defaultdict(lambda: {"src": [], "tgt": [], "vi_started": False})
STATE_TESSERACT = {"current_id": None, "mode": None, "pending_src": []}
STATE_PADDLE = {"current_id": None, "mode": None, "pending_src": []}

def process_image_to_data(img_path: str, data_map, state, ocr_model):
    if ocr_model == 'Tesseract':
        lines = tesseract_ocr(img_path)
    else:
        result = ocr_paddle.ocr(img_path, cls=True)
        if not result or result[0] is None:
            return

        blocks = result[0]
        blocks = sorted(
            blocks,
            key=lambda b: (min(p[1] for p in b[0]), min(p[0] for p in b[0]))
        )
        lines = [b[1][0].strip() for b in blocks if b and b[1] and b[1][0]]

    current_id = state["current_id"]
    mode = state["mode"]
    pending_src = state["pending_src"]

    for text in lines:
        if not text:
            continue
        if is_page_number_line(text):
            continue
        
        # print(f"Model: {ocr_model}\n{text}")

        # Title tiếng Việt -> mode=vi (lấy ID) và chỉ xử lý như title nếu extract được ID
        if looks_like_vietnamese(text):
            maybe_id = extract_id(text)
            if maybe_id:
                # Title thật
                current_id = maybe_id
                _ = data_map[current_id]
                if pending_src:
                    data_map[current_id]["src"].extend(pending_src)
                    pending_src.clear()

                mode = "vi"
                if not data_map[current_id]["vi_started"]:
                    data_map[current_id]["tgt"] = []
                    data_map[current_id]["vi_started"] = True
                
                data_map[current_id]["tgt"].append(
                    f"Bức thư viết cho chính mình số {current_id}"
                )
                continue

        # Anchor ID (ưu tiên từ tiếng Trung: 第...)
        found_id = extract_id(text)
        if found_id:
            current_id = found_id
            _ = data_map[current_id]
            if pending_src:
                data_map[current_id]["src"].extend(pending_src)
                pending_src.clear()

            mode = "zh" if is_chinese(text) else "vi"
            if mode == "vi" and (not data_map[current_id]["vi_started"]):
                data_map[current_id]["tgt"] = []
                data_map[current_id]["vi_started"] = True
            
            data_map[current_id]["src"].append(
                f"写 给 自己 的 第 {current_id} 封 信"
            )
            continue

        # Chưa có ID -> giữ tiếng Trung để chờ, bỏ qua Pinyin
        if current_id is None:
            if is_chinese(text):
                pending_src.append(text)
            continue

        # Gom nội dung theo mode
        if is_chinese(text):
            if mode == "vi":
                # Gặp tiếng Trung sau tiếng Việt -> Reset để chờ ID mới
                pending_src.append(text)
                current_id = None
                mode = "zh"
            else:
                data_map[current_id]["src"].append(text)
        else:
            if mode == "vi":
                data_map[current_id]["tgt"].append(text)

    state["current_id"] = current_id
    state["mode"] = mode
    state["pending_src"] = pending_src

In [11]:
# Main process
IMG_DIR = "../image"
OUT_DIR = "../OCR"
os.makedirs(OUT_DIR, exist_ok=True)

for p in range(START_PAGE, END_PAGE + 1):
    img_path_tesseract = os.path.join(IMG_DIR, f"{INPUT_SUBDIR}_processed/processed_PDF1-{p}.png")
    img_path_paddle = os.path.join(IMG_DIR, f"{INPUT_SUBDIR}/PDF1-{p}.png")
    if os.path.exists(img_path_paddle):
        print(f"Đang xử lý: {img_path_paddle}...")
        process_image_to_data(img_path_tesseract, DATA_MAP_TESSERACT, STATE_TESSERACT, 'Tesseract')
        process_image_to_data(img_path_paddle, DATA_MAP_PADDLE, STATE_PADDLE, 'Paddle')

rows = []
all_ids = set(DATA_MAP_PADDLE.keys()) | set(DATA_MAP_TESSERACT.keys())

def sort_key(x):
    try:
        return int(str(x))
    except:
        return str(x)

for sid in sorted(all_ids, key=sort_key):
    # Lấy Tiếng Trung (src) từ PADDLE Map
    paddle_data = DATA_MAP_PADDLE.get(sid, {"src": [], "tgt": []})
    src_text = " ".join(paddle_data["src"]).strip()
    
    # Lấy Tiếng Việt (tgt) từ TESSERACT Map
    tesseract_data = DATA_MAP_TESSERACT.get(sid, {"src": [], "tgt": []})
    tgt_text = " ".join(tesseract_data["tgt"]).strip()
    # print(f"{sid}\n")
    # print(f"{src_text}\n")
    # print(f"{tgt_text}\n")
    if src_text or tgt_text:
        rows.append({
            "src_id": sid,
            "src_lang": src_text,  # src: Paddle
            "tgt_lang": tgt_text   # tgt: Tesseract
        })
df = pd.DataFrame(rows)

# Lọc ID 
ID_START = 626
ID_END = 750
df = df[df['src_id'].str.isdigit()].copy()
df['src_id_int'] = df['src_id'].astype(int)
df = df[(df['src_id_int'] >= ID_START) & (df['src_id_int'] <= ID_END)]
df = df.sort_values('src_id_int').drop(columns=['src_id_int']).reset_index(drop=True)


Đang xử lý: ../image\PDF1/PDF1-250.png...
Đang xử lý: ../image\PDF1/PDF1-251.png...
Đang xử lý: ../image\PDF1/PDF1-252.png...
Đang xử lý: ../image\PDF1/PDF1-253.png...
Đang xử lý: ../image\PDF1/PDF1-254.png...
Đang xử lý: ../image\PDF1/PDF1-255.png...
Đang xử lý: ../image\PDF1/PDF1-256.png...
Đang xử lý: ../image\PDF1/PDF1-257.png...
Đang xử lý: ../image\PDF1/PDF1-258.png...
Đang xử lý: ../image\PDF1/PDF1-259.png...
Đang xử lý: ../image\PDF1/PDF1-260.png...
Đang xử lý: ../image\PDF1/PDF1-261.png...
Đang xử lý: ../image\PDF1/PDF1-262.png...
Đang xử lý: ../image\PDF1/PDF1-263.png...
Đang xử lý: ../image\PDF1/PDF1-264.png...
Đang xử lý: ../image\PDF1/PDF1-265.png...
Đang xử lý: ../image\PDF1/PDF1-266.png...
Đang xử lý: ../image\PDF1/PDF1-267.png...
Đang xử lý: ../image\PDF1/PDF1-268.png...
Đang xử lý: ../image\PDF1/PDF1-269.png...
Đang xử lý: ../image\PDF1/PDF1-270.png...
Đang xử lý: ../image\PDF1/PDF1-271.png...
Đang xử lý: ../image\PDF1/PDF1-272.png...
Đang xử lý: ../image\PDF1/PDF1-273

4. Hậu xử lí OCR

In [12]:
# Định nghĩa bộ lọc Regex
def is_vietnamese(text):
    if not isinstance(text, str) or text.strip() == "":
        return False
    
    # \u0020-\u007E: Bao gồm a-z, A-Z, 0-9, dấu câu chuẩn (!, @, #, ., ?, ...)
    # Phần còn lại: Các nguyên âm có dấu và chữ Đ/đ của tiếng Việt
    vietnamese_chars = (
        "àáạảãâầấậẩẫăằắặẳẵ"
        "èéẹẻẽêềếệểễ"
        "ìíịỉĩ"
        "òóọỏõôồốộổỗơờớợởỡ"
        "ùúụủũưừứựửữ"
        "ỳýỵỷỹ"
        "đ"
        "ÀÁẠẢÃÂẦẤẬẨẪĂẰẮẶẲẴ"
        "ÈÉẸẺẼÊỀẾỆỂỄ"
        "ÌÍỊỈĨ"
        "ÒÓỌỎÕÔỒỐỘỔỖƠỜỚỢỞỠ"
        "ÙÚỤỦŨƯỪỨỰỬỮ"
        "ỲÝỴỶỸ"
        "Đ"
    )
    
    # Logic: Chuỗi hợp lệ chỉ được chứa các ký tự nằm trong 2 nhóm này
    # LƯU Ý: Nếu văn bản của bạn có dấu nháy Unicode (“ ”), bạn phải thêm chúng vào pattern này
    # hoặc xử lý chúng trước khi lọc, nếu không dòng đó sẽ bị False.
    pattern = f'^[\u0020-\u007E{vietnamese_chars}]+$'
    
    return bool(re.match(pattern, text))

# --- BỔ SUNG LOGIC XÓA TRƯỚC CHỮ 写 ---
def trim_before_xie(text):
    if not isinstance(text, str):
        return text
    idx = text.find('写')
    if idx != -1:
        return text[idx:]
    return text

# Áp dụng xóa ký tự trước chữ '写' cho cột src_lang
df['src_lang'] = df['src_lang'].apply(trim_before_xie)

# ---------------------------------------
# ĐOẠN CODE GỐC CỦA BẠN TIẾP TỤC TẠI ĐÂY
# ---------------------------------------

# Drop các dòng bị thiếu (NaN) hoặc rỗng ở 1 trong 2 cột
# Chuyển chuỗi chỉ có khoảng trắng thành NaN 
df = df.replace(r'^\s*$', float('nan'), regex=True)
df.dropna(subset=['src_lang', 'tgt_lang'], inplace=True)

# Logic: Giữ lại dòng khi (Check Trung == True) và (Check Việt == True)
# Các dòng False sẽ bị loại bỏ
df_clean = df[
    df['src_lang'].apply(is_chinese) & 
    df['tgt_lang'].apply(is_vietnamese)
].copy()
df_clean.reset_index(drop=True, inplace=True)

# Xuất CSV
if not df_clean.empty:
    min_id = df_clean['src_id'].astype(int).min()
    max_id = df_clean['src_id'].astype(int).max()
else:
    min_id = max_id = 0

out_csv = os.path.join(OUT_DIR, f"PDF1_{min_id}_{max_id}.csv")
df_clean.to_csv(out_csv, index=False, encoding="utf-8-sig")
print("Đã lưu vào:", out_csv)

Đã lưu vào: ../OCR\PDF1_626_750.csv


5. Alignment cho PDF

In [3]:
SRC = 'PDF1'
FROM_ID = 626
TO_ID = 750
# INPUT_FILE = f"../data/preprocessing_data/{SRC.lower()}_{FROM_ID}_{TO_ID}.csv"
INPUT_FILE = f"../OCR/{SRC}_{FROM_ID}_{TO_ID}.csv"
OUTPUT_FILE = f"../Alignment/{SRC}_{FROM_ID}_{TO_ID}.csv"
ALIGN_THRESHOLD = 0.75

In [11]:
def get_device():
    if torch.cuda.is_available():
        return 'cuda'
    return 'cpu'

def split_sentences(text, lang='vi'):
    if not text: return []
    if lang == 'zh':
        sents = re.split(r'(?<=[。！？])\s*', text)
    else:
        sents = re.split(
            r'(?<!\d\.)(?<=[.?!])\s+(?=(?:[A-ZÀ-Ỵ"\'(]|\d{1,3}\.))',
            text
        )

    return [s.strip() for s in sents]

def align_n3(src_sents, tgt_sents, model, device, threshold):
    if not src_sents or not tgt_sents:
        return []

    # Encode
    # 1-gram (Câu đơn)
    emb_src = model.encode(src_sents, convert_to_tensor=True, device=device)
    emb_tgt = model.encode(tgt_sents, convert_to_tensor=True, device=device)
    
    # 2-gram (Gộp 2 câu)
    tgt_merged_2 = [tgt_sents[i] + " " + tgt_sents[i+1] for i in range(len(tgt_sents)-1)]
    src_merged_2 = [src_sents[i] + src_sents[i+1] for i in range(len(src_sents)-1)]
    
    emb_tgt_2 = model.encode(tgt_merged_2, convert_to_tensor=True, device=device) if tgt_merged_2 else None
    emb_src_2 = model.encode(src_merged_2, convert_to_tensor=True, device=device) if src_merged_2 else None

    # 3-gram (Gộp 3 câu) 
    tgt_merged_3 = [tgt_sents[i] + " " + tgt_sents[i+1] + " " + tgt_sents[i+2] for i in range(len(tgt_sents)-2)]
    src_merged_3 = [src_sents[i] + src_sents[i+1] + src_sents[i+2] for i in range(len(src_sents)-2)]
    
    emb_tgt_3 = model.encode(tgt_merged_3, convert_to_tensor=True, device=device) if tgt_merged_3 else None
    emb_src_3 = model.encode(src_merged_3, convert_to_tensor=True, device=device) if src_merged_3 else None

    # Tính ma trận tương đồng (similarity matrices)
    # Khởi tạo ma trận (rỗng chiều)
    def get_sim(emb1, emb2):
        if emb1 is not None and emb2 is not None:
            return util.cos_sim(emb1, emb2).cpu().numpy()
        return np.zeros((0,0))

    sim_1_1 = get_sim(emb_src, emb_tgt)
    
    sim_1_2 = get_sim(emb_src, emb_tgt_2)     # Src đơn vs Tgt gộp 2
    sim_2_1 = get_sim(emb_src_2, emb_tgt)     # Src gộp 2 vs Tgt đơn
    
    sim_1_3 = get_sim(emb_src, emb_tgt_3)     # Src đơn vs Tgt gộp 3 
    sim_3_1 = get_sim(emb_src_3, emb_tgt)     # Src gộp 3 vs Tgt đơn

    results = []
    
    # Đánh dấu câu đã dùng
    used_src = np.zeros(len(src_sents), dtype=bool)
    used_tgt = np.zeros(len(tgt_sents), dtype=bool)

    # Greedy search
    while True:
        best_score = -1
        best_type = None
        best_indices = None # (r, c)

        # Helper để check max score trong matrix
        def check_matrix(matrix, label):
            nonlocal best_score, best_type, best_indices
            if matrix.size > 0:
                r, c = np.unravel_index(matrix.argmax(), matrix.shape)
                if matrix[r, c] > best_score:
                    best_score = matrix[r, c]
                    best_type = label
                    best_indices = (r, c)

        # Kiểm tra tất cả các trường hợp
        check_matrix(sim_1_1, '1-1')
        check_matrix(sim_1_2, '1-2')
        check_matrix(sim_2_1, '2-1')
        check_matrix(sim_1_3, '1-3') 
        check_matrix(sim_3_1, '3-1') 

        if best_score < threshold:
            break

        r, c = best_indices
        valid_match = False
        
        # Logic kết quả và masking
        # Logic chung: Kiểm tra used -> Add result -> Mark used -> Mask matrix
        
        if best_type == '1-1':
            if not used_src[r] and not used_tgt[c]:
                results.append({'src': src_sents[r], 'tgt': tgt_sents[c], 'type': '1-1'})
                used_src[r] = True; used_tgt[c] = True
                valid_match = True
                # Masking: Xóa hàng r (src) và cột c (tgt) ở mọi matrix liên quan: gán -1 trực tiếp vào các vùng ảnh hưởng
                sim_1_1[r, :] = -1; sim_1_1[:, c] = -1
                if sim_1_2.size: sim_1_2[r, :] = -1; sim_1_2[:, max(0, c-1):c+1] = -1
                if sim_2_1.size: sim_2_1[max(0, r-1):r+1, :] = -1; sim_2_1[:, c] = -1
                if sim_1_3.size: sim_1_3[r, :] = -1; sim_1_3[:, max(0, c-2):c+1] = -1
                if sim_3_1.size: sim_3_1[max(0, r-2):r+1, :] = -1; sim_3_1[:, c] = -1

        elif best_type == '1-2': # Src r vs Tgt c, c+1
            if not used_src[r] and not used_tgt[c] and not used_tgt[c+1]:
                results.append({'src': src_sents[r], 'tgt': tgt_sents[c] + " " + tgt_sents[c+1], 'type': '1-2'})
                used_src[r] = True; used_tgt[c:c+2] = True
                valid_match = True
                sim_1_2[r, c] = -1 # Xóa chính nó
                sim_1_1[r, :] = -1; sim_1_1[:, c:c+2] = -1
               

        elif best_type == '2-1': # Src r, r+1 vs Tgt c
            if not used_src[r] and not used_src[r+1] and not used_tgt[c]:
                results.append({'src': src_sents[r]+src_sents[r+1], 'tgt': tgt_sents[c], 'type': '2-1'})
                used_src[r:r+2] = True; used_tgt[c] = True
                valid_match = True
                sim_2_1[r, c] = -1
                sim_1_1[r:r+2, :] = -1; sim_1_1[:, c] = -1

        elif best_type == '1-3': # Src r vs Tgt c, c+1, c+2
            if not used_src[r] and not used_tgt[c] and not used_tgt[c+1] and not used_tgt[c+2]:
                results.append({'src': src_sents[r], 'tgt': tgt_sents[c]+" "+tgt_sents[c+1]+" "+tgt_sents[c+2], 'type': '1-3'})
                used_src[r] = True; used_tgt[c:c+3] = True
                valid_match = True
                sim_1_3[r, c] = -1
                sim_1_1[r, :] = -1; sim_1_1[:, c:c+3] = -1

        elif best_type == '3-1': # Src r, r+1, r+2 vs Tgt c
            if not used_src[r] and not used_src[r+1] and not used_src[r+2] and not used_tgt[c]:
                results.append({'src': src_sents[r]+src_sents[r+1]+src_sents[r+2], 'tgt': tgt_sents[c], 'type': '3-1'})
                used_src[r:r+3] = True; used_tgt[c] = True
                valid_match = True
                sim_3_1[r, c] = -1
                sim_1_1[r:r+3, :] = -1; sim_1_1[:, c] = -1

        # Nếu không match được (do conflict đã bị dùng) -> xóa điểm này để tìm max tiếp theo
        if not valid_match:
            if best_type == '1-1': sim_1_1[r, c] = -1
            elif best_type == '1-2': sim_1_2[r, c] = -1
            elif best_type == '2-1': sim_2_1[r, c] = -1
            elif best_type == '1-3': sim_1_3[r, c] = -1
            elif best_type == '3-1': sim_3_1[r, c] = -1

        # Cập nhật kết quả
        if valid_match:
            results[-1]['score'] = float(best_score)

    return results

In [12]:
device = get_device()
print(f"Đang tải model LaBSE trên {device} (n=3)...")
model = SentenceTransformer('sentence-transformers/LaBSE', device=device)

df = pd.read_csv(INPUT_FILE)
df["src_lang"] = df["src_lang"].fillna("").astype(str)
df["tgt_lang"] = df["tgt_lang"].fillna("").astype(str)

final_data = []

print("Bắt đầu Alignment (1-1, 1-2, 2-1, 1-3, 3-1)...")
for idx, row in tqdm(df.iterrows(), total=len(df)):
    src_sents = split_sentences(row.get('src_lang', ''), lang='zh')
    tgt_sents = split_sentences(row.get('tgt_lang', ''), lang='vi')
    pairs = align_n3(src_sents, tgt_sents, model, device, ALIGN_THRESHOLD)
    
    for p in pairs:
        final_data.append({
            'src_id': row.get('src_id', ''),
            'src_lang': p['src'],
            'tgt_lang': p['tgt'],
            'score': p['score'],
            'type': p.get('type', '1-1')
        })

# Thống kê báo cáo
df_res = pd.DataFrame(final_data)
type_counts = df_res['type'].value_counts()

print("Kết quả Alignment")
print(f"Tổng số cặp: {len(df_res)}")
print(f"Phân bố cặp câu: {type_counts}")
print(f"Average Semantic Score: {df_res['score'].mean():.4f}")

# Lưu file
df_res[['src_id', 'src_lang', 'tgt_lang']].to_csv(OUTPUT_FILE, index=False, encoding='utf-8-sig')
print(f"Đã lưu file: {OUTPUT_FILE}")

Đang tải model LaBSE trên cpu (n=3)...
Bắt đầu Alignment (1-1, 1-2, 2-1, 1-3, 3-1)...


100%|██████████| 125/125 [02:20<00:00,  1.12s/it]

Kết quả Alignment
Tổng số cặp: 125
Phân bố cặp câu: type
1-1    45
1-2    32
2-1    28
1-3    17
3-1     3
Name: count, dtype: int64
Average Semantic Score: 0.8045
Đã lưu file: ../Alignment/PDF1_626_750.csv


6. Evaluation cho Alignment

In [2]:
# Config
SRC = 'PDF1'
FROM_ID = 626
TO_ID = 750
INPUT_FILE = f"../Alignment/{SRC}_{FROM_ID}_{TO_ID}.csv" 
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

In [11]:
def get_proxy_translations(df, sample_size):
    """
    Dịch câu nguồn tiếng Trung sang tiếng Việt dùng Google Translate
    để làm tham chiếu so sánh.
    """
    print(f"Đang lấy mẫu ngẫu nhiên {sample_size} cặp câu để đánh giá...")
    
    # Lấy mẫu ngẫu nhiên (random_state có thể tùy ý điều chỉnh)
    sample = df.sample(n=min(sample_size, len(df)), random_state=10).copy()

    translator = Translator()
    hypotheses = [] # Câu trong file 
    references = [] # Câu dịch máy (Làm chuẩn so sánh)
    success_count = 0
    
    for idx, row in tqdm(sample.iterrows(), total=len(sample)):
        src = str(row['src_lang'])
        tgt = str(row['tgt_lang']) # Đây là câu tiếng Việt gốc trong dataset
        
        # Bỏ qua câu quá ngắn hoặc rỗng
        if len(src) < 2 or len(tgt) < 5: 
            continue
            
        try:
            # Dịch Trung -> Việt
            translated = translator.translate(src, src='zh-cn', dest='vi').text
            
            # Câu Google dịch là Reference (Tham chiếu) và câu Tgt gốc là Hypothesis (Giả thuyết)
            references.append(translated) 
            hypotheses.append(tgt)
            
            success_count += 1
            time.sleep(0.1)
            
        except Exception as e:
            # Nếu lỗi mạng thì bỏ qua
            continue
            
    print(f"Đã dịch thành công {success_count} câu.")
    return hypotheses, references

def calculate_metrics(hypotheses, references):
    """Tính toán BLEU và BERTScore"""
    if not hypotheses:
        print("Không có dữ liệu để tính toán.")
        return

    # Tính BLEU Score
    # BLEU đánh giá độ khớp từ vựng (n-gram overlap)
    bleu = BLEU()
    bleu_score = bleu.corpus_score(hypotheses, [references])
    
    print(f"BLEU Score: {bleu_score.score:.2f}")

    # Tính BERTScore
    # BERTScore đánh giá độ tương đồng ngữ nghĩa (Semantic Similarity) dùng mô hình BERT
    P, R, F1 = score(hypotheses, references, lang='vi', verbose=False, device=DEVICE)
    
    avg_bert = F1.mean().item()
    print(f"BERTScore (F1): {avg_bert:.4f}")

    print("Ví dụ tham chiếu (Top 3):")
    for i in range(min(3, len(hypotheses))):
        print(f"Cặp {i+1}:")
        print(f"Dataset Tgt: {hypotheses[i]}")
        print(f"Google Trans: {references[i]}")

In [12]:
df = pd.read_csv(INPUT_FILE)
# Thực hiện quy trình đánh giá
SAMPLE_SIZE = int(df.shape[0] * 0.1)  # Số lượng câu lấy mẫu để đánh giá 
hyps, refs = get_proxy_translations(df, SAMPLE_SIZE)
# Tính điểm
calculate_metrics(hyps, refs)

Đang lấy mẫu ngẫu nhiên 12 cặp câu để đánh giá...


  0%|          | 0/12 [00:00<?, ?it/s]

100%|██████████| 12/12 [00:03<00:00,  3.13it/s]


Đã dịch thành công 12 câu.
BLEU Score: 19.50
BERTScore (F1): 0.8170
Ví dụ tham chiếu (Top 3):
Cặp 1:
Dataset Tgt: Nếu bạn không khuất phục thì thế giới này có thể làm gì bạn chứ.
Google Trans: Nếu bạn không nhượng bộ, thế giới này có thể làm gì bạn?
Cặp 2:
Dataset Tgt: Bức thư viết cho chính mình số 665 Cuộc sống của bạn nên vì bạn mà đa dạng sắc màu.
Google Trans: Thư 665 gửi chính mình Cuộc sống của bạn nên có sự huy hoàng của riêng bạn.
Cặp 3:
Dataset Tgt: Cuộc sống có tiến có lui, thua bất cứ thứ gì thì cũng được nhưng không được mất tâm trạng phấn khích.
Google Trans: Trong cuộc sống có những tiến bộ và rút lui, và bạn không thể đánh mất tâm trạng của mình cho dù có mất đi điều gì.
